# Capstone Project - Segmenting and Clustering Neighborhoods in Toronto - part 1,2,3

### Part 1 - Creation of Canada PostalCode DataFrame

a) Export canada postal code data from the wikipedia page to pandas dataframe

In [45]:
# Importing basic Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
# Importing libraries to query website and scrap it

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [47]:
# Query the website and return the html to the variable 'webpage'

wiki_url = " https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
webpage = urlopen(wiki_url)

In [48]:
# Parse the html in the 'webpage' variable, and store it in Beautiful Soup format

soup = BeautifulSoup(webpage, 'lxml')
type(soup)

bs4.BeautifulSoup

Since there are more than one table in this wiki page, find the class of the required table to export data

In [49]:
# look at nested structure of HTML page
# print(soup.prettify())

# find all the tables in the webpage

data_table = soup.find_all('table')
# data_table --Commenting the line since the output is too large

# find the right table

right_table=soup.find('table', class_="wikitable sortable")
# right_table --Commenting the line since the output is too large

In [50]:
# Extract all table rows of the correct table and Print the last 3 rows for sanity check

rows = right_table.find_all('tr')
print(rows[-3:])

[<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Royal York South West
</td></tr>, <tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>South of Bloor
</td></tr>, <tr>
<td>M9Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>]


In [51]:
# iterate through table rows to form a list

row_list = []
for i,row in enumerate(rows):
    row_list.append(row.find_all('td'))
print(row_list[-3:])
# type(row_list)

[[<td>M8Z</td>, <td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>, <td>Royal York South West
</td>], [<td>M8Z</td>, <td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>, <td>South of Bloor
</td>], [<td>M9Z</td>, <td>Not assigned</td>, <td>Not assigned
</td>]]


In [52]:
# extract the text without html tags

row_list = []
for i,row in enumerate(rows):
    list1 = row.find_all('td')
    str_cells = str(list1)
    cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    row_list.append(cleantext)

print(row_list[-3:])
# print(cleantext)

['[M8Z, Etobicoke, Royal York South West\n]', '[M8Z, Etobicoke, South of Bloor\n]', '[M9Z, Not assigned, Not assigned\n]']


In [53]:
# Convert list into a dataframe

toronto_df = pd.DataFrame(row_list)
toronto_df.head(10)

,0
0,[]
1,"[M1A, Not assigned, Not assigned\n]"
2,"[M2A, Not assigned, Not assigned\n]"
3,"[M3A, North York, Parkwoods\n]"
4,"[M4A, North York, Victoria Village\n]"
5,"[M5A, Downtown Toronto, Harbourfront\n]"
6,"[M5A, Downtown Toronto, Regent Park\n]"
7,"[M6A, North York, Lawrence Heights\n]"
8,"[M6A, North York, Lawrence Manor\n]"
9,"[M7A, Queen's Park, Not assigned\n]"


b) Clean and format the dataframe to get data in the required form

In [54]:
# Split and clean the df

# Drop first row
toronto_df.drop(toronto_df.index[[0]],inplace=True)
# Split the column into multiple columns
toronto_df1 = toronto_df[0].str.split(',', expand=True)
# Strip the '[,\n,]' characters from first and last columns
toronto_df1[0] = toronto_df1[0].str.strip('[')
toronto_df1[2] = toronto_df1[2].str.strip('\n]')
# Give column names
toronto_df1.columns = ['PostalCode','Borough','Neighbourhood']
# Strip any leading/trailing spaces from the values
toronto_df1['PostalCode'] = toronto_df1['PostalCode'].str.strip()
toronto_df1['Borough'] = toronto_df1['Borough'].str.strip()
toronto_df1['Neighbourhood'] = toronto_df1['Neighbourhood'].str.strip()
toronto_df1.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [55]:
# Remove all rows with 'No assigned' value for Borough

# Rows before removal
print(toronto_df1.shape)

#Value counts before removal
toronto_df1['Borough'].value_counts()

(288, 3)


Not assigned        77
Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [56]:
# Remove 'no assigned' rows

toronto_Df2 = toronto_df1[toronto_df1['Borough'] != 'Not assigned']
# reset index
toronto_Df2.reset_index(drop=True, inplace=True)

# Rows after removal
print(toronto_Df2.shape)
#Value counts after removal
# toronto_df1['Borough'].value_counts()

# See first ten rows
toronto_Df2.head(10)

(211, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [57]:
# Change the value of Neighborhood column to that of Borough column wherever no value is assigned e.g. Queen's Park Borugh(M7A)

toronto_Df2['Neighbourhood'] = toronto_Df2.apply(
    lambda row: row['Borough'] if row['Neighbourhood'] == 'Not assigned' else row['Neighbourhood'],
    axis=1
)
toronto_Df2.head(10)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [58]:
# Group neighborhoods with same values for Postalcode and Borough

toronto_Df2 = toronto_Df2.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
toronto_Df2.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [59]:
# Dimensions of the required dataframe (after joining)

toronto_Df2.shape

(103, 3)

### Part 2 - Creation of Toronto PostalCode/Geospatial DataFrame

Concat the Toronto postal code dataframe with the corresponding latitude/longitude data

In [60]:
# Import libraries

import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Set options to see full pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# install and import folium library 

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [61]:
# !pip -q install folium #if folium is not present
!pip install folium #if folium is not present
print('Installed')

import folium # map rendering library
print('Import Complete')

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Installed
Import Complete


In [62]:
# Download the geospatial dataset(i.e. latitude/longitude data) for postal codes

!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

# Read the csv file to a pandas dataframe
geo_df = pd.read_csv('Geospatial_data.csv')

# View first few rows of new df
geo_df.head()

Data downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [63]:
# Merge the formatted Toronto dataframe(Df2) and Geospatial dataframe
toronto_Df3 = pd.merge(toronto_Df2, geo_df, left_on='PostalCode', right_on='Postal Code')

# View first few rows of the merged dataframe
toronto_Df3.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [64]:
# Drop one duplicate of postal code column
toronto_Df3 = toronto_Df3.drop(['Postal Code'] , axis=1)

In [65]:
toronto_Df3.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part 3 - Explore and Cluster Toronto Neighbourhood data

Using neighbourhood data(from FourSquare) on the above dataframe, explore and cluster Toronto neighbourhood with the help of K-Means Clustering technique

a) Filter Toronto neighbourhood data from the dataframe and view its map

In [66]:
# Grouping Toronto borough data 

toronto_data = toronto_Df3[toronto_Df3['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [67]:
# Dimension of new df
print(toronto_data.shape)

# Value counts
toronto_data['Borough'].value_counts()

(38, 5)


Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [68]:
# Lets get the geographic coordinates of Toronto

address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


b) Get Toronto neighbourhood data from Foursquare API, format it to form a pandas dataframe

In [69]:
# The code was removed by Watson Studio for sharing.

In [70]:
# Function to return nearby venues for all neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 25    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
toronto_venues = getNearbyVenues(names = toronto_data['Neighbourhood'],
                                 latitudes  = toronto_data['Latitude'],
                                 longitudes = toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [72]:
# Size of the output df

print(toronto_venues.shape)
toronto_venues.head()

(727, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [73]:
# Group venues per neighbourhood

toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",25,25,25,25,25,25
Berczy Park,25,25,25,25,25,25
"Brockton,Exhibition Place,Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",25,25,25,25,25,25
Central Bay Street,25,25,25,25,25,25
"Chinatown,Grange Park,Kensington Market",25,25,25,25,25,25
Christie,15,15,15,15,15,15


In [74]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 176 uniques categories.


In [75]:
# one hot encoding for venues
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [76]:
# shape
print(toronto_onehot.shape)
  
# Group
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

(727, 177)


,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.000000,0.08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.00,0.0000,0.00,0.00,0.000000,0.000000,0.00,0.040000,0.00,0.00,0.00,0.000000,0.04,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.04,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.04,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.080000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.04,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.04,0.00,0.000000,0.04,0.04,0.00,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.04,0.00,0.00,0.000000,0.00,0.00,0.000000,0.0000,0.000000,0.00,0.0000,0.000000,0.000000,0.04,0.000000,0.00,0.000000,0.00,0.000000,0.040000,0.0000,0.000000,0.000000,0.00,0.120000,0.000000,0.040000,0.000000,0.00,0.00,0.00,0.00,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.040000,0.00,0.000000,0.00,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.04,0.08,0.00,0.04,0.000000,0.000000,0.000000,0.00,0.040000,0.000000,0.00,0.000000,

In [77]:
# Function to sort venues in desc order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    

    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
# New dataframe of neighbourhood data with values for 10 most common venues in order
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Hotel,Asian Restaurant,Noodle House,Speakeasy,Bar,Plaza,Sushi Restaurant,Coffee Shop,Concert Hall
1,Berczy Park,Farmers Market,Cocktail Bar,Beer Bar,Café,Seafood Restaurant,Vegetarian / Vegan Restaurant,Liquor Store,Basketball Stadium,Steakhouse,Clothing Store
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Convenience Store,Pet Store,Performing Arts Venue,Italian Restaurant,Grocery Store,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Park,Skate Park,Brewery,Restaurant,Burrito Place,Recording Studio,Butcher,Comic Shop,Spa
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Plane,Sculpture Garden,Boutique,Boat or Ferry,Harbor / Marina


c) Use K-Means Clustering on the above venue-neighbourhood dataframe to cluster Toronto neighbourhood

In [79]:
# Cluster the data

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])

# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

# check the first columns!
print(toronto_merged.head())
# check the last columns!
print(toronto_merged.tail())

[1 1 1 1 1 1 1 1 1 1]
  PostalCode          Borough                  Neighbourhood   Latitude  \
0        M4E     East Toronto                    The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West,Riverdale  43.679557   
2        M4L     East Toronto  The Beaches West,India Bazaar  43.668999   
3        M4M     East Toronto                Studio District  43.659526   
4        M4N  Central Toronto                  Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0     Health Food Store                 Trail   
1 -79.352188               1      Greek Restaurant        Ice Cream Shop   
2 -79.315572               1                  Park                   Gym   
3 -79.340923               1                  Café           Coffee Shop   
4 -79.388790               2              Bus Line                  Park   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \

In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the clusters

In [81]:
# Examine clusters
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Trail,Pub,Neighborhood,Yoga Studio,Creperie,Eastern European Restaurant,Dog Run,Discount Store,Diner
9,Central Toronto,0,Coffee Shop,Pub,Fried Chicken Joint,Liquor Store,Light Rail Station,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Pizza Place


In [82]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,1,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Spa,Grocery Store,Furniture / Home Store,Fruit & Vegetable Store,Juice Bar,Dessert Shop
2,East Toronto,1,Park,Gym,Sushi Restaurant,Ice Cream Shop,Fish & Chips Shop,Italian Restaurant,Fast Food Restaurant,Liquor Store,Movie Theater,Pet Store
3,East Toronto,1,Café,Coffee Shop,Bakery,Italian Restaurant,Fish Market,Bar,Middle Eastern Restaurant,Seafood Restaurant,Ice Cream Shop,Cheese Shop
5,Central Toronto,1,Gym,Breakfast Spot,Hotel,Food & Drink Shop,Park,Playground,Sandwich Place,Clothing Store,Grocery Store,Fast Food Restaurant
6,Central Toronto,1,Coffee Shop,Yoga Studio,Bagel Shop,Clothing Store,Chinese Restaurant,Rental Car Location,Salon / Barbershop,Mexican Restaurant,Dessert Shop,Diner
7,Central Toronto,1,Dessert Shop,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Gym,Gourmet Shop,Greek Restaurant,Indian Restaurant,Diner
11,Downtown Toronto,1,Restaurant,Italian Restaurant,Café,General Entertainment,Pub,Deli / Bodega,Market,Jewelry Store,Butcher,Japanese Restaurant
12,Downtown Toronto,1,Gastropub,Ice Cream Shop,Ethiopian Restaurant,Juice Bar,Breakfast Spot,Bubble Tea Shop,Salon / Barbershop,Burger Joint,Restaurant,Diner
13,Downtown Toronto,1,Coffee Shop,Park,Bakery,Mexican Restaurant,Breakfast Spot,Performing Arts Venue,Chocolate Shop,Pub,Café,Restaurant
14,Downtown Toronto,1,Café,Clothing Store,Music Venue,Pizza Place,Diner,Beer Bar,Coffee Shop,Spa,Ramen Restaurant,Burger Joint


In [83]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Bus Line,Park,Swim School,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
10,Downtown Toronto,2,Park,Playground,Trail,Creperie,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
23,Central Toronto,2,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Cuban Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner


In [84]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Playground,Tennis Court,Coffee Shop,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


In [85]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4,Garden,Ice Cream Shop,Yoga Studio,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
